In [146]:
from src.lib import state
from src.lib import ops
from random import random, sample

In [248]:
def alice_generate(num_bits=100, verbose=0):
    list_a = []
    list_x = []
    list_basis = []
    if verbose >= 1: print(f'Alice generating {num_bits} bits...')
    for _ in range(num_bits):
        a = state.bitstring(0)
        a = ops.Hadamard(1)(a)
        a_ = state.bitstring(0)
        a_ = ops.Hadamard(1)(a_)

        # measure and collapse a in correct state 
        p1, _ = ops.Measure(a, 0, 1, collapse=False)
        if p1 > random():
            _, a = ops.Measure(a, 0, 1, collapse=True)
            result_a = 1
        else:
            _, a = ops.Measure(a, 0, 0, collapse=True)
            result_a = 0

        # measure a_ and do CHadamard
        p_1, _ = ops.Measure(a_, 0, 1, collapse=False)
        if p_1 > random():
            x = ops.Hadamard(1)(a)
            result_a_ = 1
            if result_a == 0:
                result_x = '+'
            else:
                result_x = '-'
        else:
            x = a
            result_a_ = 0
            result_x = result_a
        if verbose >= 2:
            print(f'Alice: a = {result_a} and a_ = {result_a_} --> x = {result_x}.')
        list_a.append(a)
        list_x.append(x)
        list_basis.append(result_a_)
    return list_a, list_x, list_basis

In [261]:
def eve(list_x_states, perc_spy = 0.5, verbose = 0):
    new_l_x_states = []
    knowledge_eve = []
    num_spied = int(len(list_x_states) * perc_spy)
    chosen_idx = sample(range(len(list_x_states)), num_spied)
    if verbose >= 1: print(f'Eve is catches {len(list_x_states)} bits and measures {num_spied}...')
    for i, s in enumerate(list_x_states): 
        if i in chosen_idx: 
            e_ = state.bitstring(0)
            e_ = ops.Hadamard(1)(e_)
    
            # measure e_ and do CHadamard
            p_1, _ = ops.Measure(e_, 0, 1, collapse=False)
            if p_1 > random():
                e = ops.Hadamard(1)(s)
                result_e_ = 1
            else:
                e = s
                result_e_ = 0
    
            # measure and collapse e
            p1, _ = ops.Measure(e, 0, 1, collapse=False)
            if p1 > random():
                _, e = ops.Measure(e, 0, 1, collapse=True)
                result_e = 1
            else:
                _, e = ops.Measure(e, 0, 0, collapse=True)
                result_e = 0
            
            # Return into original s (CHadamard)
            if result_e_:
                s = ops.Hadamard(1)(e)
            else:
                s = e
            if verbose >= 2:
                print(f'Eve: e_ = {result_e_} --> e = {result_e}.')
            knowledge_eve.append((result_e, result_e_))
            new_l_x_states.append(s)
            
        else: 
            knowledge_eve.append(-1)
            new_l_x_states.append(s)
    return new_l_x_states, knowledge_eve

In [262]:
def bob(list_x, verbose=0):
    list_basis = []
    list_b = []
    if verbose >= 1: print(f'Bob measuring {len(list_x)} bits from Alice...')
    for x in list_x:
        b_ = state.bitstring(0)
        b_ = ops.Hadamard(1)(b_)

        # measure b_ and do CHadamard
        p_1, _ = ops.Measure(b_, 0, 1, collapse=False)
        if p_1 > random():
            b = ops.Hadamard(1)(x)
            result_b_ = 1
        else:
            b = x
            result_b_ = 0

        # measure and collapse b
        p1, _ = ops.Measure(b, 0, 1, collapse=False)
        if p1 > random():
            _, b = ops.Measure(b, 0, 1, collapse=True)
            result_b = 1
        else:
            _, b = ops.Measure(b, 0, 0, collapse=True)
            result_b = 0

        if verbose >= 2:
            print(f'Bob: b_ = {result_b_} --> b = {result_b}.')
        list_b.append(b)
        list_basis.append(result_b_)
    return list_b, list_basis

In [263]:
def valid_invalid(list_a_basis, list_b_basis, verbose=0):
    valid_state_idx = []
    for idx, (b_a, b_b) in enumerate(zip(list_a_basis, list_b_basis)):
        if b_a == b_b:
            valid_state_idx.append(idx)
        if verbose >= 2:
            print(f'Bit with index {idx} is {"" if b_a == b_b else "not "}valid, because {b_a} vs {b_b}.')
    if verbose >= 1: print(f'Number of valid bits between Alice and Bob: {len(valid_state_idx)}')
    return valid_state_idx

In [291]:
def eve_extract_information(knowledge_eve, list_valid, list_basis, verbose=0):
    num_corr_bits = 0
    for idx in list_valid:
        # if eve spied that valid bit
        if knowledge_eve[idx] != -1: 
            # if basis from A == B == E
            if knowledge_eve[idx][1] == list_basis[idx]: 
                num_corr_bits += 1
                if verbose >= 2: print(f'Bit with index {idx} was spied correctly with {knowledge_eve[idx][0]}')
    if verbose >= 1: print(f'Eve spied {num_corr_bits} of {len([1 for x in list_valid if knowledge_eve[x] != -1])} valid spied bits of all spied {len(list_basis) - knowledge_eve.count(-1)} bits correctly.')
    return num_corr_bits

In [292]:
def sacrifice(states_a, states_b, perc_sacrifice = 0.5, threshold = 0.75, verbose=False):
    clear_channel = False
    
    num_valid = 0 
    num = 0
    pairs = list(zip(states_a, states_b))
    
    num_sec = int(perc_sacrifice * len(pairs))
    if verbose >= 1: print(f'Alice and Bob sacrificing {num_sec} of {len(pairs)} bits...')
    for s_a, s_b in sample(pairs, num_sec): 
        num += 1
        
        p_a, _ = ops.Measure(s_a, 0, 0, collapse=False)
        p_b, _ = ops.Measure(s_b, 0, 0, collapse=False)
        if round(p_a, 2) == round(p_b, 2):
            num_valid += 1
        if verbose >= 2: print(f'Compared {s_a} to {s_b} and its {"valid" if round(p_a, 2) == round(p_b, 2) else "invalid"}.')
    if num_valid / num > threshold: 
        clear_channel = True 
    if verbose >= 1: print(f'{num_valid} from {num} are correct and so channel is {"probably clear" if clear_channel else "maybe spied"}!')
    return clear_channel

In [293]:
l_a_states, l_x_states, l_a_basis = alice_generate(1000, verbose=1)

l_x_states, k_eve = eve(l_x_states, perc_spy=0.5, verbose=1)

l_b_states, l_b_basis = bob(l_x_states, verbose=1)

valid_idx = valid_invalid(l_a_basis, l_b_basis, verbose=1)

n_spied_bits = eve_extract_information(k_eve, valid_idx, l_a_basis, verbose=1)

l_a_states_valid = [l_a_states[i] for i in valid_idx]
l_b_states_valid = [l_b_states[i] for i in valid_idx]
c = sacrifice(l_a_states_valid, l_b_states_valid, perc_sacrifice=0.5, threshold=0.9, verbose=1)

print(c)

Alice generating 1000 bits...
Eve is catches 1000 bits and measures 500...
Bob measuring 1000 bits from Alice...
Number of valid bits between Alice and Bob: 519
Eve spied 129 of 269 valid spied bits of all spied 500 bits correctly.
Alice and Bob sacrificing 259 of 519 bits...
227 from 259 are correct and so channel is maybe spied!
False
